In [16]:
import numpy as np
import mdtraj as md
import pandas as pd
from Bio.PDB import Superimposer, PDBParser, PDBIO
from Bio import PDB
import warnings
warnings.filterwarnings("ignore")
import glob

In [17]:
# Loading reference and Alphafold structures - change path for AF2 or AF3
path_AF2 = 'Data/Aligned/*.pdb'
path_AF3 = 'Data/AF3_Aligned/*.pdb'

sample_files = []
for filename in glob.glob(path_AF3):
    sample_files.append(filename)

reference_files = []
for sample_file in sample_files:
    reference_file_name = sample_file.split('/')[-1].split('_')[0].lower()
    for filename in glob.glob('Data/TM_only_final/*'):
        if filename.endswith('.pdb') and filename.split('/')[-1].split('_')[0].lower() == reference_file_name:
            reference_files.append(filename)

In [18]:
from Bio.PDB import PDBParser
import mdtraj as md

# Function to get the TM regions from the reference PDB files
def get_top_subsequences(pdb_file, num_subsequences=7):
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    subsequences = []
    count = 0

    for model in structure:
        for chain in model:
            current_res_num = None
            current_subsequence = []
            first_res_num = None

            for residue in chain:
                res_num = residue.get_full_id()[3][1]
                if current_res_num is None or res_num != current_res_num + 1:
                    if current_subsequence:
                        subsequences.append(current_subsequence)
                    current_subsequence = []
                    if first_res_num is None:
                        first_res_num = res_num
                if residue.has_id("CA"):
                    current_subsequence.append((residue.get_resname(), residue["CA"].get_vector(), count))
                    current_res_num = res_num
                    count += 1
            if current_subsequence:
                subsequences.append(current_subsequence)

    subsequences.sort(key=len, reverse=True)
    top_subsequences = subsequences[:num_subsequences]
    top_subsequences.sort(key=lambda subseq: subseq[0][2])

    return top_subsequences

# Function to find the corresponding subsequence in the Alphafold structure
def find_corresponding_gen_subsequence(ref_subseq, gen_structure):
    ref_length = len(ref_subseq)
    best_match = None
    gen_sequence = []

    for model in gen_structure:
        for chain in model:
            for residue in chain:
                if residue.has_id("CA"):
                    gen_sequence.append((residue.get_resname(), residue["CA"].get_vector(), residue.get_full_id()[3][1]-1))

    for i in range(len(gen_sequence) - ref_length + 1):
        matching_length = sum(1 for ref_aa, gen_aa in zip(ref_subseq, gen_sequence[i:i+ref_length]) if ref_aa[0] == gen_aa[0])
        if best_match is None or matching_length > best_match[1]:
            best_match = (i, matching_length, gen_sequence[i:i+ref_length])

    return best_match[2] 

# Function to get the indices required to calculate H3-H6 distance 
def get_indices_from_subsequences(subsequences):
 
    third_to_last_index = subsequences[2][-3][2]
    third_index = subsequences[5][2][2]
    return third_to_last_index, third_index

def compare_subsequences(ref_pdb, gen_pdb):
    parser = PDBParser()
    ref_structure = parser.get_structure("ref", ref_pdb)
    gen_structure = parser.get_structure("gen", gen_pdb)

    ref_sequences = get_top_subsequences(ref_pdb)
    gen_subseq = [find_corresponding_gen_subsequence(ref_subseq, gen_structure) for ref_subseq in ref_sequences]

    ref_indices = get_indices_from_subsequences(ref_sequences)
    gen_indices = get_indices_from_subsequences(gen_subseq)

  
    ref_aa1, ref_aa2 = ref_sequences[2][-3][0], ref_sequences[5][2][0]
    gen_aa1, gen_aa2 = gen_subseq[2][-3][0], gen_subseq[5][2][0]
    if ref_aa1 == gen_aa1 and ref_aa2 == gen_aa2:
        
        return ref_indices, gen_indices
    else:
        return None, None
 

# Function to calculate H3-H6 distances
def calculate_distances(ref_pdb_list, gen_pdb_list):
    ref_distances = []
    gen_distances = []
    protein_names = []

    for ref_pdb, gen_pdb in zip(ref_pdb_list, gen_pdb_list):
        protein_name = ref_pdb.split('/')[-1].split('_')[0].lower()
        ref_indices, gen_indices = compare_subsequences(ref_pdb, gen_pdb)
        if ref_indices and gen_indices:
            traj_ref = md.load(ref_pdb)
            traj_gen = md.load(gen_pdb)
            ref_distance = md.compute_contacts(traj_ref, [[ref_indices[0], ref_indices[1]]], scheme = 'ca')[0][0][0]
            gen_distance = md.compute_contacts(traj_gen, [[gen_indices[0], gen_indices[1]]], scheme = 'ca')[0][0][0]
            ref_distances.append(ref_distance)
            gen_distances.append(gen_distance)
            protein_names.append(protein_name)

    return protein_names,ref_distances, gen_distances


ref_pdb_list = reference_files
gen_pdb_list = sample_files


protein_names,ref_distances, gen_distances = calculate_distances(ref_pdb_list, gen_pdb_list)

In [19]:
# Calculating delta H3-H6 and storing the results in a dataframe
delta_h3_h6 = abs(np.array(ref_distances) - np.array(gen_distances))
df_results = pd.DataFrame({"Protein names": protein_names,"Reference Distances": ref_distances, "Generated Distances": gen_distances, "Delta H3-H6": delta_h3_h6})
df_results.sort_values(by = 'Protein names', inplace = True)


In [20]:
#  Saving final results - change the path for AF2 and AF3
deform_path_AF2 = 'Data/deform_results_AF2.csv'
deform_path_AF3 = 'Data/deform_results_AF3.csv'
df_labels_deform = pd.read_csv(deform_path_AF3)
df_labels_deform.sort_values(by=['Protein'], inplace=True)

df_results.reset_index(drop=True, inplace=True)
df_labels_deform.reset_index(drop=True, inplace=True)


df_labels_deform['Delta H3-H6'] = df_results['Delta H3-H6']

df_labels_deform.to_csv('Data/deform_h3_h6_results_AF3.csv', index = False)

In [21]:
print(df_labels_deform)

   Protein  Overlap_Ratio  Activity_level Class  Average Distance  Delta H3-H6
0    5ht1b       0.995595              24     A          1.134164     0.131718
1    5ht2a       1.000000             100     A          3.858857     0.270579
2    5ht2b       0.995614              41     A          1.061522     0.039201
3    5ht2c       0.995595               1     A          1.367736     0.079109
4     aa1r       1.000000             100     A          1.990629     1.126907
..     ...            ...             ...   ...               ...          ...
70   pth1r       1.000000             100    B1          2.433251     0.219180
71   s1pr1       0.995434               1     A          2.015047     0.287751
72     smo       1.000000               0     F          4.263313     0.036702
73    ta2r       0.995516              41     A          1.099221     0.140276
74     v2r       1.000000             100     A          5.541794     0.031125

[75 rows x 6 columns]
